In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('udf').getOrCreate()

24/07/12 13:23:50 WARN Utils: Your hostname, MZC01-HYUCKSANGCHO.local resolves to a loopback address: 127.0.0.1; using 10.90.9.46 instead (on interface en0)
24/07/12 13:23:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/12 13:23:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/12 13:23:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
trip_files = "/Users/mzc01-hyucksangcho/Downloads/trips/*"
zone_file = "/Users/mzc01-hyucksangcho/Downloads/taxi_zone_lookup.csv"

trips_df = spark.read.parquet(f"file:///{trip_files}", inferSchema=True, header = True)
zone_df = spark.read.csv(f"file:///{zone_file}", inferSchema=True, header = True)

In [16]:
trips_df.createOrReplaceTempView("trips")
zone_df.createOrReplaceTempView("zone")

In [18]:
query = """
SELECT
    t.VendorID as vendor_id,
    TO_DATE(t.tpep_pickup_datetime) as pickup_date,
    TO_DATE(t.tpep_dropoff_datetime) as dropoff_date,
    HOUR(t.tpep_pickup_datetime) as pickup_time,
    HOUR(t.tpep_dropoff_datetime) as dropoff_time,
    t.passenger_count,
    t.trip_distance,
    t.fare_amount,
    t.tip_amount,
    t.tolls_amount,
    t.total_amount,
    t.payment_type,
    t.tpep_pickup_datetime as pickup_datetime,
    pz.Zone as pickup_zone,
    dz.Zone as dropoff_zone
from
    trips t
    LEFT JOIN
        zone pz
    ON
        t.PULocationID = pz.LocationID
    LEFT JOIN
        zone dz
    ON
        t.DOLocationID = dz.LocationID
"""
comb_df = spark.sql(query)
comb_df.show()


+---------+-----------+------------+-----------+------------+---------------+-------------+-----------+----------+------------+------------+------------+-------------------+--------------------+--------------------+
|vendor_id|pickup_date|dropoff_date|pickup_time|dropoff_time|passenger_count|trip_distance|fare_amount|tip_amount|tolls_amount|total_amount|payment_type|    pickup_datetime|         pickup_zone|        dropoff_zone|
+---------+-----------+------------+-----------+------------+---------------+-------------+-----------+----------+------------+------------+------------+-------------------+--------------------+--------------------+
|        1| 2021-06-01|  2021-06-01|          0|           0|            1.0|          0.9|        5.0|       2.2|         0.0|        11.0|           1|2021-06-01 00:13:26|Penn Station/Madi...|        Clinton West|
|        1| 2021-06-01|  2021-06-01|          0|           1|            1.0|         23.0|       61.5|       0.0|        6.55|        7

In [19]:
comb_df.show()

+---------+-----------+------------+-----------+------------+---------------+-------------+-----------+----------+------------+------------+------------+-------------------+--------------------+--------------------+
|vendor_id|pickup_date|dropoff_date|pickup_time|dropoff_time|passenger_count|trip_distance|fare_amount|tip_amount|tolls_amount|total_amount|payment_type|    pickup_datetime|         pickup_zone|        dropoff_zone|
+---------+-----------+------------+-----------+------------+---------------+-------------+-----------+----------+------------+------------+------------+-------------------+--------------------+--------------------+
|        1| 2021-06-01|  2021-06-01|          0|           0|            1.0|          0.9|        5.0|       2.2|         0.0|        11.0|           1|2021-06-01 00:13:26|Penn Station/Madi...|        Clinton West|
|        1| 2021-06-01|  2021-06-01|          0|           1|            1.0|         23.0|       61.5|       0.0|        6.55|        7

In [22]:
comb_df.select('pickup_date').describe().show()

+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    max|
+-------+

